<a href="https://colab.research.google.com/github/casualcomputer/llm_google_colab/blob/main/setup_ollma_google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Host LLM on colab (with ollama) with port forwarding to public internet + access colab-hosted model via ollama API

In [1]:
from google.colab import userdata #user to access token

In [ ]:
!curl https://ollama.ai/install.sh | sh

!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers

!pip install pyngrok
from pyngrok import ngrok
ngrok.set_auth_token(userdata.get('ngrok_token')) #secret saved in a variable "ngrok_token" in colab "Secrets"

import os
import asyncio

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

# Function to run a specified command as a subprocess
async def run_process(cmd):
    # Print the command that is starting
    print('>>> starting', *cmd)

    # Start the subprocess and redirect stdout and stderr to asyncio subprocess pipes
    p = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,  # Capture standard output
        stderr=asyncio.subprocess.PIPE   # Capture standard errors
    )

    # Function to handle the output from stdout and stderr
    async def pipe(lines):
        async for line in lines:
            # Decode and strip each line of output and print it
            print(line.decode('utf-8').strip())

    # Concurrently process both stdout and stderr
    await asyncio.gather(
        pipe(p.stdout),
        pipe(p.stderr),
    )

# Clear the output of the current Jupyter notebook cell
clear_output()

# This coroutine runs two subprocesses: one for the Ollama server and one for the ngrok tunnel
async def main():
    await asyncio.gather(
        run_process(['ollama', 'serve']),  # Command to start the Ollama server
        run_process(['ngrok', 'http', '--log', 'stderr', '11434']),  # Command to start ngrok to expose port 11434
    )

# Run the main function in the asyncio event loop
await main()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10941    0 10941    0     0  25815      0 --:--:-- --:--:-- --:--:-- 25865
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-

Command prompt on local laptops' cmd

```
curl https://ollama.ai/install.sh | sh
export OLLAMA_HOST=(https://fe64-34-106-204-48.ngrok-free.app/)
ollama run dolphin-mistral
```





# Host LLM on colab (with ollama) and access the LLM on colab with ollama API

In [2]:
# Install ollama: The Ollama API is now available at 127.0.0.1:11434
! curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
#!curl -X POST http://127.0.0.1:11434/api/generate -d '{"model": "llama3", "prompt": "Why is the sky blue?"}'

In [ ]:
#!curl -X POST http://localhost:11434/api/generate -d '{"model": "llama3", "prompt": "Why is the sky blue?"}'

curl: (7) Failed to connect to localhost port 11434 after 0 ms: Connection refused


In [16]:
!pip install ngrok
!pip install pyngrok==4.1.1

In [17]:
from google.colab import userdata #user to access token

# Get your ngrok token from your ngrok account:
# https://dashboard.ngrok.com/get-started/your-authtoken
import threading
import time
import os
import asyncio
from pyngrok import ngrok
import threading
import queue
import time
from threading import Thread

# Get your ngrok token from your ngrok account:
# https://dashboard.ngrok.com/get-started/your-authtoken
token= userdata.get('ngrok_token')
ngrok.set_auth_token(token)

# set up a stoppable thread (not mandatory, but cleaner if you want to stop this later
class StoppableThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(StoppableThread, self).__init__(*args, **kwargs)
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def is_stopped(self):
        return self._stop_event.is_set()

def start_ngrok(q, stop_event):
    try:
        # Start an HTTP tunnel on the specified port
        public_url = ngrok.connect(11434)
        # Put the public URL in the queue
        q.put(public_url)
        # Keep the thread alive until stop event is set
        while not stop_event.is_set():
            time.sleep(1)  # Adjust sleep time as needed
    except Exception as e:
        print(f"Error in start_ngrok: {e}")

In [18]:
# Create a queue to share data between threads
url_queue = queue.Queue()

# Start ngrok in a separate thread
ngrok_thread = StoppableThread(target=start_ngrok, args=(url_queue, StoppableThread.is_stopped))
ngrok_thread.start()

In [19]:
# Wait for the ngrok tunnel to be established
while True:
    try:
        public_url = url_queue.get()
        if public_url:
            break
        print("Waiting for ngrok URL...")
        time.sleep(1)
    except Exception as e:
        print(f"Error in retrieving ngrok URL: {e}")

print("Ngrok tunnel established at:", public_url)

KeyboardInterrupt: 

In [9]:
import os
import asyncio

# NB: You may need to set these depending and get cuda working depending which backend you are running.
# Set environment variable for NVIDIA library
# Set environment variables for CUDA
os.environ['PATH'] += ':/usr/local/cuda/bin'
# Set LD_LIBRARY_PATH to include both /usr/lib64-nvidia and CUDA lib directories
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/cuda/lib64'

async def run_process(cmd):
    print('>>> starting', *cmd)
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    # define an async pipe function
    async def pipe(lines):
        async for line in lines:
            print(line.decode().strip())

        await asyncio.gather(
            pipe(process.stdout),
            pipe(process.stderr),
        )

    # call it
    await asyncio.gather(pipe(process.stdout), pipe(process.stderr))

In [10]:
import asyncio
import threading

async def start_ollama_serve():
    await run_process(['ollama', 'serve'])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
thread.start()

>>> starting ollama serve


In [13]:
! export OLLAMA_HOST=http://9126-34-106-204-48.ngrok-free.app/

In [20]:
!ollama pull llama2

Error: could not connect to ollama app, is it running?


## Method 1: ollama cmd

In [19]:
!ollama run llama2  #method 1

Error: could not connect to ollama app, is it running?


## Method 2: curl and API

In [18]:
!curl -X POST http://9126-34-106-204-48.ngrok-free.app/api/generate -d '{ "model": "llama2", "prompt":"Good Morning" , "stream": false }'

## Method 3: Python and API

In [16]:
# python code

import requests

# URL for the API endpoint
url = 'http://9126-34-106-204-48.ngrok-free.app//api/generate'

# Function to make the POST request
def make_request(prompt):
    data = {
        "model": "llama2",
        "prompt": prompt,
        "stream": False
    }
    response = requests.post(url, json=data)
    if response.status_code == 200:
        return response.json().get("response", "No response received")
    else:
        return f"Error: {response.status_code}, {response.text}"

# Continuous chat loop
while True:
    # Get user input for the prompt
    prompt = input("You: ")

    # Check if the user wants to exit
    if prompt.lower() == 'exit':
        print("Exiting chat.")
        break

    # Make the request
    response = make_request(prompt)

    # Print the response
    print("Bot:", response)


You: hi there. how are ya
Bot: Error: 404, <!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com

KeyboardInterrupt: Interrupted by user

In [ ]:
#! pip install pipenv && pipenv install

To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing dependencies from Pipfile.lock (e4eef2)...


In [ ]:
!git clone https://github.com/casualcomputer/local-rag.git #gitclone local rag's interface folder

fatal: destination path 'local-rag' already exists and is not an empty directory.


In [ ]:
!streamlit run /local-rag/main.py &>/content/logs.txt & npx localtunnel --port 8501 &curl ipv4.icanhazip.com

34.172.141.48
npx: installed 22 in 4.168s
your url is: https://warm-hornets-open.loca.lt
